In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

## Load dataset

In [2]:
path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00597/garments_worker_productivity.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
df.sample(n=10)

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
325,1/19/2015,Quarter3,sweing,Monday,2,0.70,22.94,1006.0,10170,38,0.0,0,0,56.5,0.750518
487,1/28/2015,Quarter4,finishing,Wednesday,12,0.75,4.08,NaN,1080,0,0.0,0,0,9.0,0.755556
1008,3/2/2015,Quarter1,sweing,Monday,1,0.65,26.66,1527.0,6840,65,0.0,0,0,57.0,0.800580
553,2/1/2015,Quarter1,sweing,Sunday,9,0.75,29.12,1282.0,6960,45,0.0,0,0,58.0,0.750593
817,2/17/2015,Quarter3,sweing,Tuesday,9,0.65,18.79,2120.0,5520,0,0.0,0,1,51.0,0.650134
347,1/20/2015,Quarter3,sweing,Tuesday,5,0.70,20.40,1352.0,10260,50,0.0,0,0,57.0,0.700064
571,2/2/2015,Quarter1,finishing,Monday,12,0.75,4.08,NaN,1080,0,0.0,0,0,9.0,0.740444
259,1/15/2015,Quarter3,finishing,Thursday,4,0.35,4.30,NaN,3600,0,0.0,0,0,20.0,0.988636
557,2/1/2015,Quarter1,sweing,Sunday,6,0.65,18.79,881.0,3960,26,0.0,0,0,33.0,0.651007
596,2/3/2015,Quarter1,finishing,Tuesday,4,0.80,4.30,NaN,1440,0,0.0,0,0,12.0,0.537500


In [5]:
df.shape

(1197, 15)

Bộ dữ liệu có 1197 sample và 15 feature.

## Data wrangling

Kiểm tra kiểu dữ liệu của từng cột.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int64  
 5   targeted_productivity  1197 non-null   float64
 6   smv                    1197 non-null   float64
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int64  
 9   incentive              1197 non-null   int64  
 10  idle_time              1197 non-null   float64
 11  idle_men               1197 non-null   int64  
 12  no_of_style_change     1197 non-null   int64  
 13  no_of_workers          1197 non-null   float64
 14  actual_productivity    1197 non-null   float64
dtypes: f

Chuyển kiểu của 'date' về dạng datetime.

In [7]:
df['date'] = pd.to_datetime(df['date'])

In [8]:
df['date'].sample(n=5)

1186   2015-03-11
1118   2015-03-08
679    2015-02-09
166    2015-01-10
623    2015-02-05
Name: date, dtype: datetime64[ns]

Xem các giá trị khác nhau của các biến kiểu object.

In [9]:
df['quarter'].unique()

array(['Quarter1', 'Quarter2', 'Quarter3', 'Quarter4', 'Quarter5'],
      dtype=object)

In [10]:
df['department'].unique()

array(['sweing', 'finishing ', 'finishing'], dtype=object)

Có 2 department là 'sweing' (bị viết sai chính tả) và 'finishing' (có 1 từ bị dư dấu cách cuối). Sửa lại cho đúng.

In [11]:
df['department'] = df['department'].apply(lambda x: 'sewing' if x == 'sweing' else 'finishing')

In [12]:
df['department'].unique()

array(['sewing', 'finishing'], dtype=object)

In [13]:
df['day'].unique()

array(['Thursday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday'],
      dtype=object)

Trong df['day'] không có 'Friday', các công nhân được nghỉ ngày thứ sáu.

In [14]:
df['team'].unique()

array([ 8,  1, 11, 12,  6,  7,  2,  3,  9, 10,  5,  4], dtype=int64)

Biến 'team' mang tính phân loại (categorical), vì ý nghĩa của nó là số thứ tự của team chứ không mang ý nghĩa về mặt giá trị số. Có thể chuyển về 'object' cũng được hoặc giữ nguyên cũng được (nhưng phân tích như biến categorical).

Kiểm tra giá trị null.

In [15]:
df.isna().sum(axis=0)

date                       0
quarter                    0
department                 0
day                        0
team                       0
targeted_productivity      0
smv                        0
wip                      506
over_time                  0
incentive                  0
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers              0
actual_productivity        0
dtype: int64

Feature 'wip' có tới 506 giá trị null, ta điền giá trị khuyết bằng phương pháp kNN với thư viện sklearn.

In [16]:
from sklearn.impute import KNNImputer

imputer = KNNImputer()
df[df.columns[4:]] = imputer.fit_transform(df.iloc[:,4:])

In [17]:
df['wip'].isna().sum()

0

In [18]:
df['wip']

0       1108.0
1        994.4
2        968.0
3        968.0
4       1170.0
         ...  
1192     994.4
1193     994.4
1194     994.4
1195     632.8
1196     926.0
Name: wip, Length: 1197, dtype: float64

## Wrapping in a single function

In [19]:
def data_loader(path='https://archive.ics.uci.edu/ml/machine-learning-databases/00597/garments_worker_productivity.csv',
                preprocessing=False):
    df = pd.read_csv(path)
    if preprocessing == True:
        df['date'] = pd.to_datetime(df['date'])
        df['department'] = df['department'].apply(lambda x: 'sewing' if x == 'sweing' else 'finishing')
        from sklearn.impute import KNNImputer
        imputer = KNNImputer()
        df[df.columns[4:]] = imputer.fit_transform(df.iloc[:,4:])
    return df

In [20]:
df1 = data_loader(preprocessing=True)

In [21]:
df1.dtypes

date                     datetime64[ns]
quarter                          object
department                       object
day                              object
team                            float64
targeted_productivity           float64
smv                             float64
wip                             float64
over_time                       float64
incentive                       float64
idle_time                       float64
idle_men                        float64
no_of_style_change              float64
no_of_workers                   float64
actual_productivity             float64
dtype: object

In [22]:
df1.isna().sum(axis=0)

date                     0
quarter                  0
department               0
day                      0
team                     0
targeted_productivity    0
smv                      0
wip                      0
over_time                0
incentive                0
idle_time                0
idle_men                 0
no_of_style_change       0
no_of_workers            0
actual_productivity      0
dtype: int64